## Import

In [15]:
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [21]:
# Load historical data
file_path = "./data/Anleihen und Indizes historische Zinsen.xlsx"
df_anleihen = pd.read_excel(file_path, sheet_name="Anleihen", skiprows=1)
#df_etf = pd.read_excel(file_path, sheet_name="Indizes")

df_anleihen = df_anleihen.drop(df_anleihen.columns[14], axis=1)
df_anleihen.columns = ["date", "australia", "canada", "france", "germany", "japan", "korea", "spain", "switzerland", "uk", "us", "non_oecd", "china_1", "india", "china_2"]


df_anleihen.head(10)

,date,australia,canada,france,germany,japan,korea,spain,switzerland,uk,us,non_oecd,china_1,india,china_2
0,Time period,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,China
1,1954,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.401667,NaN,NaN,NaN,NaN
2,1955,NaN,3.189167,NaN,NaN,NaN,NaN,NaN,2.982500,NaN,2.816667,NaN,NaN,NaN,NaN
3,1956,NaN,3.605833,NaN,NaN,NaN,NaN,NaN,3.125000,NaN,3.182500,NaN,NaN,NaN,NaN
4,1957,NaN,4.125000,NaN,7.516667,NaN,NaN,NaN,3.667500,NaN,3.647500,NaN,NaN,NaN,NaN
5,1958,NaN,4.115833,NaN,6.783333,NaN,NaN,NaN,3.153333,NaN,3.315833,NaN,NaN,NaN,NaN
6,1959,NaN,5.048333,NaN,5.766667,NaN,NaN,NaN,3.076667,NaN,4.333333,NaN,NaN,NaN,NaN
7,1960,NaN,5.188334,5.663333,6.425000,NaN,NaN,NaN,3.092500,5.855834,4.116667,NaN,NaN,NaN,NaN
8,1961,NaN,5.058333,5.519166,5.875000,NaN,NaN,NaN,2.955833,6.264167,3.882500,NaN,NaN,NaN,NaN
9,1962,NaN,5.099167,5.434166,5.908333,NaN,NaN,NaN,3.124167,5.764167,3.945833,NaN,NaN,NaN,NaN
